# Model based analysis of Risk and Ambiguity

Using the utility function to try and asses Risk and ambiguity aversion using the formula from 

SV = (probability − β (Ambiguity / 2)] × Value ** α 


In [1]:
import pandas as pd
import numpy as np

import pymc3 as pm
import arviz as az

import matplotlib.pyplot as plt

load monatery data add subject sequential number starting from 0

In [2]:
db = pd.read_csv('Aging/RiskandAmbiguity/mon.csv')
db['subn'] = db['sub'].rank(method='dense').astype(int)-1
db.head()

,Unnamed: 0,choice,value,risk,ambiguity,sub,catch,subn
0,0,0.0,5,0.75,0.00,11,0.0,1
1,1,0.0,8,0.50,0.24,11,0.0,1
2,2,0.0,25,0.50,0.74,11,0.0,1
3,3,0.0,25,0.50,0.74,11,0.0,1
4,4,0.0,25,0.25,0.00,11,0.0,1


transfer values and subjects to arrays

In [3]:
sub_idx = np.asarray(db['subn'])
value = np.asarray(db['value'])
n_subs = max(sub_idx)+1
n_subs

51

### model 1: risk aversion without ambiguity

In [4]:
with pm.Model() as Risk:

    # Priors for unknown model parameters
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    
    α = pm.Normal('α', 0, 1,  shape = n_subs)
    γ = pm.Normal('γ', 0, 10, shape = n_subs)
    
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    svLotto = value ** α[sub_idx] * db.risk
    svRef = 5 ** α[sub_idx]
    mu =  1/( 1 + np.exp(γ[sub_idx] * (svLotto - svRef)))
    
    #y_hat = 1/(1+np.exp(γ * (safe-bet))) + eps
    
    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=db.choice)

    trace = pm.sample(2000, return_inferencedata=False)

/home/nachshon/anaconda3/envs/wmdm/lib/python3.8/site-packages/pymc3/model.py:1755: ImputationWarning: Data in Y_obs contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Y_obs_missing, sigma, γ, α, intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 187 seconds.
There were 500 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.4016365778917362, but should be close to 0.8. Try to increase the number of tuning steps.
There were 389 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.08163485153955428, but should be close to 0.8. Try to increase the number of tuning steps.
There were 745 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.40759268815986754, but should be close to 0.8. Try to increase the number of tuning steps.
There were 406 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The esti

### model 2: risk and ambiguity

In [5]:
with pm.Model() as RiskAmb:

    # Priors for unknown model parameters
    α = pm.Normal('α', 1, 1, shape = n_subs)
    β = pm.Normal('β', 0, 1, shape = n_subs)
    γ = pm.Normal('γ', 0, 10, shape = n_subs)
    
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Expected value of outcome
    svLotto = value ** α[sub_idx] * -(β[sub_idx] * (db.ambiguity/2) - db.risk)
    svRef = 5 ** α[sub_idx]
    mu =  1/( 1 + np.exp(γ[sub_idx] * (svLotto - svRef)))
       
    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=db.choice)

    trace = pm.sample(2000, return_inferencedata=False)

/home/nachshon/anaconda3/envs/wmdm/lib/python3.8/site-packages/pymc3/model.py:1755: ImputationWarning: Data in Y_obs contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Y_obs_missing, sigma, γ, β, α]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 394 seconds.
There were 1102 divergences after tuning. Increase `target_accept` or reparameterize.
There were 907 divergences after tuning. Increase `target_accept` or reparameterize.
There were 839 divergences after tuning. Increase `target_accept` or reparameterize.
There were 1526 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
